In [1]:
%matplotlib inline

import numpy as np
import scipy
import scipy.io
import scipy.sparse as sp
import matplotlib.pyplot as plt
import surprise
%load_ext autoreload
%autoreload 2

# Co-clustering

In [9]:
from surprise.model_selection import GridSearchCV
from surprise import CoClustering
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split

# path to dataset file
file_path = "../data/data_surprise.csv"

reader = Reader(line_format='user item rating', sep=',', skip_lines=1)

data = Dataset.load_from_file(file_path, reader=reader)

param_grid = {'n_cltr_u': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'n_cltr_i': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
              'n_epochs': [15]}

gs = GridSearchCV(CoClustering, param_grid, measures=['rmse', 'mae'], cv=2)
gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

1.0101742819292434
{'n_cltr_u': 1, 'n_cltr_i': 1, 'n_epochs': 5}


In [11]:
from surprise import accuracy

trainset, testset = train_test_split(data, test_size=.10)

algo = gs.best_estimator['rmse']

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 1.0063


1.0062558689645806

### Ratings that have to be predicted (items and users specified in sample_submission file)

In [9]:
from data_helpers import read_csv_sample

INPUT_PATH = "../data/sample_submission.csv"
ids = read_csv_sample(INPUT_PATH)

### Write your craziest ML algorithm

### Save output for submission

In [4]:
from data_helpers import create_csv_submission

OUTPUT_PATH = "../data/submission.csv"
create_csv_submission(ids, predictions, OUTPUT_PATH)
print("File submission.csv ready to be submitted !")

SyntaxError: invalid syntax (<ipython-input-4-b53775eec35b>, line 3)